In [1]:
import os
import PIL
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Utility Functions

In [2]:
#Function for showing images
def imshow(img):
    #img = img / 2 + 0.5  
    #print(img)
    #plt.imshow can take in images with 0-1 (floats) or 0-255 (int)
    plt.imshow(np.transpose(img, (1, 2, 0))) 
    #print(img)
    #print(max(img[0][0]))
    print(np.mean(img[0][2]))
    print(np.std(img[0][2]))
    #print(max(img[0][1]))
    #print(max(img[0][2]))
#It's best to keep the data processing code separate from the class loader
def to_tensor_and_normalize(imagepil): #Done with testing
    """Convert image to torch Tensor and normalize using the ImageNet training
    set mean and stdev taken from
    https://pytorch.org/docs/stable/torchvision/models.html.
    Why the ImageNet mean and stdev instead of the PASCAL VOC mean and stdev?
    Because we are using a model pretrained on ImageNet."""
    #Think the reason for introducing normalisation is because of the imagenet weights
    #ChosenTransforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
    #            torchvision.transforms.Normalize(mean=[0.6236, 0.5118, 0.4264],std=[0.3545, 0.2692, 0.3376]),])


    #This straight up just transforms 0-255 to 0-1
    ChosenTransforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])#, torchvision.transforms.Normalize(mean=[0.3, 0.3, 0.3],std=[0.05, 0.05, 0.05])])
    #ChosenTransforms = torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])#[torchvision.transforms.ToTensor()])
    """ChosenTransforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                      lambda x: x*1.666666666666666666667,
                                                      ])"""
    return ChosenTransforms(imagepil)

# Class for loading in my simulated image training data

In [3]:
class SimData(Dataset):
    def __init__(self, setname):
        '''The sim input images and output images are 128 x 128 x 3'''
        self.setname = setname
        assert setname in ['train','val','test']
        
        #Define where to load in the dataset
        overall_dataset_dir = os.path.join(os.path.join(os.getcwd(),'load_dataset'), 'data')
        #input images
        self.selected_dataset_dir = os.path.join(overall_dataset_dir,setname)
        
        #output images
        self.selected_dataset_output_dir = os.path.join(overall_dataset_dir,setname+"_outputs")
        
        #E.g. self.all_filenames = ['im1.jpg',..,'imN.jpg'] when setname=='train'
        #Loads in the input images from the training folder
        self.all_filenames = os.listdir(self.selected_dataset_dir)
        self.all_filenames_output = os.listdir(self.selected_dataset_output_dir)
        
    def __len__(self):
        """Return the total number of examples in this split, e.g. if
        self.setname=='train' then return the total number of examples
        in the training set"""
        return len(self.all_filenames)
    
    def __getitem__(self,idx):
        """Return the example at index [idx]. The example is a dict with keys
        'data' (value: Tensor for an RGB image) and labels are also images"""
        #For the inputs
        selected_filename = self.all_filenames[idx]
        #print(selected_filename)
        #test= self.all_filenames_output[idx]
        #print(test)
        imagepil = PIL.Image.open(os.path.join(self.selected_dataset_dir,selected_filename)).convert('RGB')
        
        #For the outputs
        selected_filename_output = self.all_filenames_output[idx]
        imagepil_output = PIL.Image.open(os.path.join(self.selected_dataset_output_dir,
                                                      selected_filename_output)).convert('RGB')
        
        #convert image to Tensor/normalize
        image = to_tensor_and_normalize(imagepil)
        image_output = to_tensor_and_normalize(imagepil_output)
        
        
        sample = {'data':image, #preprocessed image, for input into NN
                  'label':image_output,
                  'img_idx':idx}
        
        return sample

# Now to load in the dataset

In [4]:
# #Basically load the created class in
# train_dataset = SimData("train")

# #Prepare data loaders
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=0, shuffle = False)

# Display the loaded in images for a sanity check lol

In [5]:
# dataiter = iter(train_loader)
# #For some reason this is needed
# output = next(dataiter)
# images = output['data'].numpy()
# labels = output['label'].numpy()
# #print(labels)
# #Rather than this
# #images, labels, _ = next(dataiter)
# #images = images.numpy() # convert images to numpy for display


# #Plot the images
# print("Input Images (Sensor Data)")
# fig, axes = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(12,4))
# for idx in np.arange(5):
#     ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
#     imshow(images[idx])
#     #ax.set_title(classes[labels[idx]])
# plt.show()

# #Output images
# print("Shape output Images")
# fig, axes = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(12,4))
# for idx in np.arange(5):
#     ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
#     imshow(labels[idx])
#     #ax.set_title(classes[labels[idx]])
# plt.show() 
    

In [6]:
# print(images[0].shape)
# print(labels[0].shape)
# print(images[0])
# print(np.max(images[0][1]))
# print(np.mean(images[0][1]))

#  Conv autoencoder

In [7]:
class ConvAutoencoder(nn.Module):
    def __init__(self, layer_disp = False):
        super(ConvAutoencoder, self).__init__()
        self.layer_disp = layer_disp
        #Encoder
        #nn.Conv2(in_channels, out_channels, kernel_size, stride, padding)
        self.conv1 = nn.Conv2d(in_channels=3, 
                               out_channels=128,
                               kernel_size=2, 
                               stride = 1,
                               padding = 'same')
        self.conv2 = nn.Conv2d(in_channels=128, 
                               out_channels=256,
                               kernel_size=2, 
                               stride = 2,
                               padding = 0)
        
        self.conv3 = nn.Conv2d(in_channels=256, 
                               out_channels=256,
                               kernel_size=4, 
                               stride = 4,
                               padding = 0)
        
        
        
        self.pool = nn.MaxPool2d(2,2)
        
       
        #Decoder
        #nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)
        self.t_conv1 = nn.ConvTranspose2d(in_channels=256, 
                               out_channels=128,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        self.t_conv2= nn.ConvTranspose2d(in_channels=128, 
                               out_channels=128,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        self.t_conv3= nn.ConvTranspose2d(in_channels=128, 
                               out_channels=64,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        
        self.t_conv4= nn.ConvTranspose2d(in_channels=64, 
                               out_channels=64,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        self.t_conv5= nn.ConvTranspose2d(in_channels=64, 
                               out_channels=32,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        '''self.t_conv6= nn.ConvTranspose2d(in_channels=32, 
                               out_channels=32,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)'''
       
        self.t_convout = nn.ConvTranspose2d(in_channels=32, 
                               out_channels=3,
                               kernel_size=2, 
                               stride = 2,
                               padding=0)
        #self.upsample = nn.functional.interpolate(scale_factor = 2)


    def forward(self, x):
        en_layer1 = F.relu(self.conv1(x))   
        en_layer1 = self.pool(en_layer1)
        en_layer2 = F.relu(self.conv2(en_layer1))
        en_layer2 = self.pool(en_layer2)
        en_layer3 = F.relu(self.conv3(en_layer2))
        en_layer3 = self.pool(en_layer3)

        #de_layer0 = F.relu(self.t_conv0(en_layer4)) 
        de_layer1 = F.relu(self.t_conv1(en_layer3))   
        #de_layer1 = nn.functional.interpolate(de_layer1,scale_factor = 2)
        de_layer2= F.relu(self.t_conv2(de_layer1)) 
        de_layer3= F.relu(self.t_conv3(de_layer2))
        de_layer4= F.relu(self.t_conv4(de_layer3))
        de_layer5= F.relu(self.t_conv5(de_layer4))
        #de_layer6= F.relu(self.t_conv6(de_layer5))
        
        #de_layerout = torch.sigmoid(self.t_conv3(de_layer2))
        de_layerout = F.relu(self.t_convout(de_layer5))
        #de_layerout = self.t_conv5(de_layer4)
        
        if self.layer_disp:
            print("input", x.shape)
            print("en_layer1",en_layer1.shape)
            #print("en_layer1a",en_layer1a.shape)
            print("en_layer2",en_layer2.shape)
            #print("en_layer2a",en_layer2a.shape)
            print("en_layer3",en_layer3.shape)
            #print("en_layer3a",en_layer3a.shape)
            #print("en_layer4",en_layer4.shape)
            #print("en_layer5",en_layer5.shape)
            
            #print("de_layer0",de_layer0.shape)
            print("de_layer1",de_layer1.shape)
            #print("de_layer1a",de_layer1a.shape)
            print("de_layer2",de_layer2.shape)
            print("de_layer3",de_layer3.shape)
            print("de_layer4",de_layer4.shape)
            print("de_layer5",de_layer5.shape)
            #print("de_layer6",de_layer6.shape)
            #print("de_layer2a",de_layer2a.shape)
            #print("de_layer3",de_layerou.shape)
            #print("de_layer3a",de_layer3a.shape)
            #print("de_layer4",de_layer4.shape)
            print("de_layerout",de_layerout.shape)
              
        return de_layerout

In [8]:
#Now to intialise the model and shit
model = ConvAutoencoder(layer_disp = False).to(device)
model = ConvAutoencoder(layer_disp = True).to(device)

#Defining the loss function between the input and the output
#criterion = nn.BCELoss()
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
#Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1*1e-3)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma=0.5)

#This is just for testing and seeing the outputs of each convolutional layer
dummy = model(torch.empty(3, 128, 128).to(device))
#from torchviz import make_dot
#make_dot(dummy, params=dict(list(model.named_parameters()))).render("autoencoder", format="png")




input torch.Size([3, 128, 128])
en_layer1 torch.Size([128, 64, 64])
en_layer2 torch.Size([256, 16, 16])
en_layer3 torch.Size([256, 2, 2])
de_layer1 torch.Size([128, 4, 4])
de_layer2 torch.Size([128, 8, 8])
de_layer3 torch.Size([64, 16, 16])
de_layer4 torch.Size([64, 32, 32])
de_layer5 torch.Size([32, 64, 64])
de_layerout torch.Size([3, 128, 128])


/home/parth/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv2d(input, weight, bias, self.stride,


# Train the model

In [9]:
# from livelossplot import PlotLosses

# n_epochs = 10000 #Number of epochs

# liveloss = PlotLosses()
# logs = {}


# for epoch in range(1, n_epochs+1):
#     # monitor training loss
#     train_loss = 0.0

#     #Training
#     for data in train_loader:
#         #For some reason this is needed
#         output = data
#         #print(output)
#         images = output['data']
#         labels = output['label']
#         #images, _ = data #Don't care about the labels
#         images = images.to(device) 
#         labels = labels.to(device) 
#         optimizer.zero_grad()
#         outputs = model(images)
#         #outputs = model(labels) #to test if the autoencoder works
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()*images.size(0)
          
#     train_loss = train_loss/len(train_loader)
#     logs['loss'] = train_loss
    
#     #print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
#     predictions = outputs
#     predictions = predictions.detach().cpu().numpy()
#     imshow(predictions[0])
#     if not(epoch%50) or epoch ==0:
#         plt.show()

#     liveloss.update(logs)
#     liveloss.send()
    

# Display the predictions

In [10]:

#Basically load the created class in
test_dataset = SimData("test")

model.load_state_dict(torch.load("/home/parth/UROP_2023/deblur.pth", map_location=torch.device('cpu')))
model.eval()

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=40, num_workers=0, shuffle = False)
#Batch of test images
#Batch of test images
with torch.no_grad():
    dataiter = iter(test_loader)
    #images, labels = next(dataiter)
    output = next(dataiter)
    images = output['data'].to(device) #these are the inputs
    labels = output['label'].to(device)
    #labels = output['label'].numpy() #again labels aren't needed

    #Sample outputs
    #predictions = model(labels) #test just the atoencoder ibit
    predictions = model(images) #output of the network
    images = images.detach().cpu().numpy() #convert the images back to numpy arrays
    labels = labels.detach().cpu().numpy() #convert the images back to numpy arrays

    #convert back to appropriate numpy array and shit
    print(images.shape)
    print(predictions.shape)
    predictions = predictions.detach().cpu().numpy()
    #predictions = predictions.view(batch_size, 3, 32, 32)


    #Original Images
    print("Sensor Input")
    #fig, axes = plt.subplots(nrows=1, ncols=8, sharex=True, sharey=True, figsize=(12,4))
    for idx in np.arange(39):
        #ax = fig.add_subplot(1, 8, idx+1, xticks=[], yticks=[])
        #imshow(images[idx])

        input = np.transpose(images[idx], (1, 2, 0))
        input = np.array(255*input, dtype = 'uint8')
        input = cv2.cvtColor(input, cv2.COLOR_RGB2BGR)
        os.makedirs('inputs', exist_ok=True) 
        base_path = os.path.join('inputs',"im{}.jpg".format(idx)) #training outputs
        cv2.imwrite(base_path,input)
        #ax.set_title(classes[labels[idx]])
    plt.show()
    
    # #Original Images
    # print("Real Shape Images")
    # fig, axes = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(12,4))
    # for idx in np.arange(5):
    #     ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[])
    #     imshow(labels[idx])
    #     #ax.set_title(classes[labels[idx]])
    # plt.show()

    #Reconstructed Images
    print('Reconstructed Predicted Shape Images')
    #fig, axes = plt.subplots(nrows=1, ncols=8, sharex=True, sharey=True, figsize=(12,4))
    for idx in np.arange(39):
        #ax = fig.add_subplot(1, 8, idx+1, xticks=[], yticks=[])
        #imshow(predictions[idx])

        pred = np.transpose(predictions[idx], (1, 2, 0))
        pred = np.array(255*pred, dtype = 'uint8')
        pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
        os.makedirs('predictions', exist_ok=True) 
        base_path = os.path.join('predictions',"im{}.jpg".format(idx)) #training outputs
       
        cv2.imwrite(base_path,pred) 
        #ax.set_title(classes[labels[idx]])
    plt.show() 

input torch.Size([39, 3, 128, 128])
en_layer1 torch.Size([39, 128, 64, 64])
en_layer2 torch.Size([39, 256, 16, 16])
en_layer3 torch.Size([39, 256, 2, 2])
de_layer1 torch.Size([39, 128, 4, 4])
de_layer2 torch.Size([39, 128, 8, 8])
de_layer3 torch.Size([39, 64, 16, 16])
de_layer4 torch.Size([39, 64, 32, 32])
de_layer5 torch.Size([39, 32, 64, 64])
de_layerout torch.Size([39, 3, 128, 128])
(39, 3, 128, 128)
torch.Size([39, 3, 128, 128])
Sensor Input


IndexError: index 39 is out of bounds for axis 0 with size 39

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="qnNaV5QQtZevwcJxsA5Y")
project = rf.workspace().project("digit-braille")
classifier = project.version(3).model

number = 0

# infer on a local image
print(classifier.predict("/home/parth/UROP_2023/autoencoder/inputs/im{}.jpg".format(number), confidence=10, overlap=30).json())

# visualize your prediction
classifier.predict("/home/parth/UROP_2023/autoencoder/inputs/im{}.jpg".format(number), confidence=10, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

ImportError: cannot import name 'appengine' from 'urllib3.contrib' (/home/parth/.local/lib/python3.10/site-packages/urllib3/contrib/__init__.py)